# Day 2 - Lab 2: Documenting Key Decisions with ADRs (Solution)

**Objective:** Use an LLM as a research assistant to compare technical options and synthesize the findings into a formal, version-controlled Architectural Decision Record (ADR).

**Introduction:**
This solution notebook provides the complete prompts and explanations for the ADR generation lab. It demonstrates how to use an LLM for comparative research and then synthesize that research into a structured, formal document.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

In [2]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): The ADR Template

**Explanation:**
This prompt asks the LLM to generate a standard markdown template for an ADR. The key is to be specific about the sections required (`Title`, `Status`, `Context`, `Decision`, `Consequences`), which guides the LLM to produce a well-structured and reusable template.

In [6]:
adr_template_prompt = """You are a principal engineer who champions clear documentation. Generate a concise, reusable markdown template for an Architectural Decision Record (ADR).

The template must include the following sections:
- # Title: [A short, descriptive title for the decision]
- **Status:** [Proposed | Accepted | Deprecated | Superseded]
- ## Context
  - [Describe the problem, the driving forces, and the constraints.]
- ## Decision
  - [State the chosen solution clearly and concisely.]
- ## Consequences
  - [List the positive outcomes, negative trade-offs, and any future work required.]"""

print("--- Generating ADR Template ---")
adr_template_content = get_completion(adr_template_prompt, client, model_name, api_provider)
print(adr_template_content)

if adr_template_content:
    save_artifact(adr_template_content, "templates/adr_template.md")

--- Generating ADR Template ---
```markdown
# Title: [A short, descriptive title for the decision]

**Status:** [Proposed | Accepted | Deprecated | Superseded]

## Context
- [Describe the problem, the driving forces, and the constraints.]

## Decision
- [State the chosen solution clearly and concisely.]

## Consequences
- [List the positive outcomes, negative trade-offs, and any future work required.]
```

### Usage Notes:
- **Title:** Summarize the decision in a way that can be easily referenced.
- **Status:** Reflect the current state of the decision. Update this status as the decision evolves through its lifecycle.
- **Context:** Provide sufficient background to understand why this decision is necessary. Include information about stakeholders, technical constraints, and any relevant historical context.
- **Decision:** Clearly articulate the decision that has been made. Include any relevant technical details or specifications that support the decision.
- **Consequences:** Highlight t

### Challenge 2 (Intermediate): AI-Assisted Research

**Explanation:**
This prompt leverages the LLM's vast training data to perform a comparative analysis. By instructing it to be an "unbiased research assistant" and asking for "pros and cons for each," we guide the model to provide a balanced view rather than a simple recommendation. This produces a more valuable and objective input for our own decision-making process.

In [8]:
db_research_prompt = """You are an unbiased research assistant. Your task is to provide a balanced technical comparison for a software development team.

For the use case of a new hire onboarding tool that needs a semantic search feature, compare and contrast the following two approaches:

1.  **Approach 1:** Using PostgreSQL with the `pgvector` extension.
2.  **Approach 2:** Using a specialized, dedicated vector database (e.g., ChromaDB, FAISS, Weaviate).

Please provide a summary of the pros and cons for each approach. Consider factors like operational complexity, cost, query flexibility, and scalability for a small-to-medium sized enterprise application.
"""

print("--- Researching Database Options ---")
db_research_output = get_completion(db_research_prompt, client, model_name, api_provider)
print(db_research_output)

--- Researching Database Options ---
When choosing an approach for implementing a semantic search feature in a new hire onboarding tool, the decision between using PostgreSQL with the `pgvector` extension and a specialized vector database involves evaluating several factors such as operational complexity, cost, query flexibility, and scalability. Below is a balanced comparison of both approaches:

### Approach 1: PostgreSQL with `pgvector` Extension

**Pros:**

1. **Unified System:** 
   - Using PostgreSQL with `pgvector` allows for managing both traditional relational data and vector embeddings within a single database system, reducing the need for integrating multiple systems.

2. **Operational Simplicity:**
   - If the team is already using PostgreSQL, extending it with `pgvector` minimizes the learning curve and operational overhead compared to adopting a new database technology.
   - PostgreSQL is a well-known and widely adopted database system, which makes it easier to find resou

### Challenge 3 (Advanced): Synthesizing the ADR

**Explanation:**
This prompt demonstrates a powerful synthesis task. We provide the LLM with two key inputs: unstructured information (the research) and a desired structure (the template). The agent's job is to merge them, creating a polished, formal document. This is a repeatable pattern for turning raw analysis into professional documentation. By assigning the persona of a Staff Engineer, we guide the LLM to adopt a formal and authoritative tone suitable for an official project artifact.

In [10]:
adr_template = load_artifact("templates/adr_template.md")

synthesis_prompt = f"""You are a Staff Engineer responsible for documenting key architectural decisions.

Your task is to populate the provided ADR template to formally document the decision to **use PostgreSQL with the pgvector extension** for our project.

Use the research provided below to fill in the 'Context' and 'Consequences' sections of the template. Be thorough and objective, summarizing the key points from the research.

--- ADR TEMPLATE ---
{adr_template}
--- END TEMPLATE ---

--- RESEARCH CONTEXT ---
{db_research_output}
--- END CONTEXT ---

The final ADR should be complete and ready for review.
"""

print("--- Synthesizing Final ADR ---")
if adr_template and 'db_research_output' in locals() and db_research_output:
    final_adr = get_completion(synthesis_prompt, client, model_name, api_provider)
    print(final_adr)
    save_artifact(final_adr, "artifacts/adr_001_database_choice.md")
else:
    print("Skipping ADR synthesis because template or research is missing.")

--- Synthesizing Final ADR ---
```markdown
# Title: Adoption of PostgreSQL with pgvector Extension for Semantic Search

**Status:** Proposed

## Context
- Our project aims to implement a semantic search feature within a new hire onboarding tool. This feature requires efficient handling of vector embeddings for similarity searches. 
- We must decide between using PostgreSQL with the `pgvector` extension or a specialized vector database (e.g., ChromaDB, FAISS, Weaviate).
- Key considerations include operational complexity, cost, query flexibility, and scalability.
- The team is already familiar with PostgreSQL, which is part of our current tech stack, thus minimizing the learning curve and operational overhead.
- Cost constraints favor leveraging existing infrastructure without incurring additional expenses for new hardware or cloud services.
- The anticipated dataset size and query volume for the onboarding tool are moderate, making PostgreSQL with `pgvector` a potentially viable option

## Lab Conclusion

Well done! You have used an LLM to automate a complex but critical part of the architectural process. You leveraged its vast knowledge base for research and then used it again for synthesis, turning raw analysis into a formal, structured document. This `adr_001_database_choice.md` file now serves as a permanent, valuable record for anyone who works on this project in the future.

> **Key Takeaway:** The pattern of **Research -> Synthesize -> Format** is a powerful workflow. You can use an LLM to gather unstructured information and then use it again to pour that information into a structured template, creating high-quality, consistent documentation with minimal effort.